### Setting up SQL database with Py using sqlite3



In [ ]:
! pip install sqlite3

#### Tables required for inflation analysis:

After taking overview I found below metrics play critical role into how any economist generally draw analysis on state of inflation for any given country in given time frame.

- Consumer Price Index
- Producer Price Index
- Gross Domestic Product
- Employment Cost Index
- Commodity Price Index
- Housing Price Index

Due to constraints of data availability and scope of this project at this stage, I have considered Gross Development Product(GDP) to draw analysis.

In [ ]:
import sqlite3

# Connect to SQLite and create a database
conn = sqlite3.connect("inflation_records.db")
cursor = conn.cursor()

# Create a table named "GDP"
cursor.execute('''
    CREATE TABLE IF NOT EXISTS GDP (
        country TEXT,
        code TEXT,
        year INTEGER,
        inflation INTEGER
    )
''')

# Commit changes and close connection
conn.commit()
conn.close()


##### Add GDP data to table

In [ ]:
import csv
 
# Connect to SQLite and create a database
conn = sqlite3.connect("inflation_records.db")
cursor = conn.cursor()

# Read data from the inflation-GDP CSV file
with open('inflation-gdp.csv') as f:
    reader = csv.DictReader(f)
    data = list(reader)
 
# Insert data into the GDP table
for row in data:
    cursor.execute("INSERT INTO GDP(country, code, year, inflation) VALUES (?, ?, ?, ?)",
                   (row['Country'], row['Country Code'], row['Year'], row['Inflation']))
 
# Commit the changes and close connection
conn.commit()
conn.close()

*Display GDP data*

In [ ]:
import sqlite3
 
# Connect to the database
conn = sqlite3.connect('inflation_records.db')  
cursor = conn.cursor()
 
# Execute a SELECT query on the "GDP" table
cursor.execute('SELECT DISTINCT country FROM GDP')
 
# Fetch all rows
rows = cursor.fetchall()
 
# Display the data
for row in rows:
    print(row)
 
# Close the database connection
conn.close()

*Display GDP data using matplotlib*

In [ ]:
! pip install matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data // just for now using csv
data = pd.read_csv('inflation-gdp.csv')

# Basic Data Exploration
print("Data Overview:")
print(data.head())

# Core Functionality
def filter_data(country=None, start_year=None, end_year=None):
    filtered_data = data.copy()

    if country:
        filtered_data = filtered_data[filtered_data['Country'] == country]

    if start_year:
        filtered_data = filtered_data[filtered_data['Year'] >= start_year]

    if end_year:
        filtered_data = filtered_data[filtered_data['Year'] <= end_year]

    return filtered_data

# Example: Filter data for a specific country and range of years
filtered_data = filter_data(country='Arab World', start_year=1970, end_year=1980)

# Data Visualization
def plot_inflation_trend(data):
    plt.plot(data['Year'], data['Inflation'], marker='o')
    plt.title(f'Inflation Trend for {data["Country"].iloc[0]} ({data["Year"].min()} to {data["Year"].max()})')
    plt.xlabel('Year')
    plt.ylabel('Inflation')
    plt.show()

# Example: Plot inflation for the filtered data
plot_inflation_trend(filtered_data)


##### Add database functionalities

*Create New Entry to DB*

In [ ]:
import sqlite3

# Function to establish a database connection
def connect_to_database():
    return sqlite3.connect("inflation_records.db")

# Create new entry in the GDP table
def create_entry(country, code, year, inflation):
    conn = connect_to_database()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO GDP(country, code, year, inflation) VALUES (?, ?, ?, ?)",
                   (country, code, year, inflation))
    conn.commit()
    conn.close()

# Example Usage:
# Create Entry
create_entry("ExampleCountry", "EX", 2022, 5.0)

*Read Entry from DB*

In [ ]:
import sqlite3

# Function to establish a database connection
def connect_to_database():
    return sqlite3.connect("inflation_records.db")

# Read entries from the GDP table based on filters
def read_entries(country=None, start_year=None, end_year=None):
    conn = connect_to_database()
    cursor = conn.cursor()

    query = "SELECT * FROM GDP WHERE 1"
    params = []

    if country:
        query += " AND country = ?"
        params.append(country)

    if start_year:
        query += " AND year >= ?"
        params.append(start_year)

    if end_year:
        query += " AND year <= ?"
        params.append(end_year)

    cursor.execute(query, tuple(params))
    entries = cursor.fetchall()

    conn.close()
    return entries


# Read Entries
entries = read_entries(country='ExampleCountry')
print("Read Entries:", entries)

*Update Entry in DB*

In [ ]:
import sqlite3

# Function to establish a database connection
def connect_to_database():
    return sqlite3.connect("inflation_records.db")

# Update entry in the GDP table
def update_entry(country, year, new_inflation):
    conn = connect_to_database()
    cursor = conn.cursor()
    cursor.execute("UPDATE GDP SET inflation = ? WHERE country = ? AND year = ?",
                   (new_inflation, country, year))
    conn.commit()
    conn.close()

# Update Entry
update_entry("ExampleCountry", 2022, 6.0)

*Delete Entry in DB*

In [ ]:
import sqlite3

# Function to establish a database connection
def connect_to_database():
    return sqlite3.connect("inflation_records.db")

# Delete entry from the GDP table
def delete_entry(country, year):
    conn = connect_to_database()
    cursor = conn.cursor()
    cursor.execute("DELETE FROM GDP WHERE country = ? AND year = ?", (country, year))
    conn.commit()
    conn.close()

# Delete Entry
delete_entry("ExampleCountry", 2022)
